In [ ]:
import torch, transformers, accelerate
import bitsandbytes as bnb

# sanity checks
bnb_version = getattr(bnb, "__version__", "installed (no __version__ attribute)")
print("Torch:", torch.__version__)
print("Transformers:", transformers.__version__)
print("Accelerate:", accelerate.__version__)
print("Bitsandbytes:", bnb_version)
print("\nCUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device:", torch.cuda.get_device_name(0))
    print("CUDA version:", torch.version.cuda)
else:
    print("Running on CPU only")


Torch: 2.5.1+cu121
Transformers: 4.56.2
Accelerate: 1.10.1
Bitsandbytes: 0.48.0

CUDA available: True
Device: NVIDIA GeForce RTX 3060 Laptop GPU
CUDA version: 12.1


### Hugging Face Token id is Hidded

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from huggingface_hub import login

login("*")

In [19]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2b",
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation="sdpa",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
input_text = "In a special sequence, each number is generated by the total number of letters when spelling the previous number in English. If the sequence begins with the number 4, what is the fifth number in the sequence?. Report the fifth number"
input_ids = tokenizer(input_text, return_tensors="pt").to(model.device)

outputs = model.generate(**input_ids, max_new_tokens = 200, cache_implementation="static")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In a special sequence, each number is generated by the total number of letters when spelling the previous number in English. If the sequence begins with the number 4, what is the fifth number in the sequence?. Report the fifth number in the sequence.

Answer:

Step 1/2
The first number in the sequence is 4. The second number in the sequence is 4 + 4 = 8. The third number in the sequence is 8 + 4 = 12. The fourth number in the sequence is 12 + 4 = 16. The fifth number in the sequence is 16 + 4 = 20.

Step 2/2
Therefore, the fifth number in the sequence is 20.


### Gemma 2B works fine on local device.
### No errors (like gpu out of memory) were encountered as of now. 
### For fast access here we are using 4bit Quantisation

## Hence Gemma can possibly be a candidate for our AI Agent